In [131]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [132]:
# get list of urls for players stats
players_url = 'https://www.basketball-reference.com/players/'
r = requests.get(players_url)
bs = BeautifulSoup(r.text)

In [133]:
# get all letter links
letter_links = bs.find('ul', {'class': 'page_index'}).find_all('a')
letter_links = ['https://www.basketball-reference.com' + link.get('href') for link in letter_links if len(link.text) == 1]
print(len(letter_links))

25


In [134]:
# get all player links from letter links
player_links = []

for letter_link in letter_links:
    r = requests.get(letter_link)
    bs = BeautifulSoup(r.text)
    player_tags = bs.find('tbody').find_all('th', {'scope': 'row', 'class': 'left'})
    player_links += ['https://www.basketball-reference.com' + player_tag.find('a').get('href') for player_tag in player_tags if player_tag.find('strong')]

print(len(player_links))

859


In [135]:
# build dataframe of all players' stats by season

appended_data = []

for player_link in tqdm(player_links):
    dfs = pd.read_html(player_link)
    df = dfs[1]

    r = requests.get(player_link)
    bs = BeautifulSoup(r.text)
    name = bs.find("div", {"id": "info", "class": "players"}).find("h1").text
    name = name.split('\n')[1]

    df['Name'] = name
    df['Exp'] = df.index + 1

    appended_data.append(df)

nba = pd.concat(appended_data)

100%|██████████| 859/859 [07:48<00:00,  1.83it/s]


In [145]:
first = nba.pop('Exp')
nba.insert(0, 'Exp', first)

first = nba.pop('Name')
nba.insert(0, 'Name', first)

nba = nba.dropna(subset=['Age'])

nba = nba.reset_index(drop=True)

In [146]:
nba.head()

Name  Exp   Season   Age   Tm   Lg Pos     G    GS    MP  ...  \
0      Álex Abrines    1  2016-17  23.0  OKC  NBA  SG     5     0  16.0  ...   
1      Álex Abrines    2  2017-18  24.0  OKC  NBA  SG     6     0  18.3  ...   
2  Precious Achiuwa    1  2020-21  21.0  MIA  NBA  PF  61.0   4.0  12.1  ...   
3  Precious Achiuwa    2  2021-22  22.0  TOR  NBA   C  73.0  28.0  23.6  ...   
4  Precious Achiuwa    3  2022-23  23.0  TOR  NBA  PF   1.0   0.0  18.0  ...   

     FT%  ORB  DRB  TRB  AST  STL  BLK  TOV   PF   PTS  
0   0.75  0.4  1.4  1.8  0.8  0.0  0.0  0.0  1.6   4.8  
1    1.0  0.5  2.2  2.7  0.3  0.8  0.3  0.2  1.8   4.0  
2  0.509  1.2  2.2  3.4  0.5  0.3  0.5  0.7  1.5   5.0  
3  0.595  2.0  4.5  6.5  1.1  0.5  0.6  1.2  2.1   9.1  
4    0.5  1.0  4.0  5.0  0.0  0.0  0.0  0.0  1.0  10.0  

[5 rows x 32 columns]

In [148]:
nba.shape

(4637, 32)

In [150]:
nba.Name.nunique()

859